# Lecture 2: In-class activity

## First, get Jupyter notebook up and running.

If you've made it here, you've done this step!  

## Second, familiarize yourself with python.  
As a first pass, you may want to play around with the examples in the cells below.  Hit Shift+Enter to evaluate a cell.

In [ ]:
print("Hello World!")  # Hello world is really easy in python!

: 

In [ ]:
A = [6,4,3,8,5] # A is a list.  
print(A) # you can print it out!

In [ ]:
A[0] # lists are zero-indexed. 

In [ ]:
# slicing up lists:
print(A[2:4]) # this is the list [A[2],A[3]] (it doesn't include A[4])
print(A[2:])  # this notation starts with A[2] and goes to the end
print(A[:4])  # this starts at the beginning and goes up until A[3]
print(A[:])   # this just returns a copy of the whole list

In [ ]:
len(A) # get the length of a list

In [ ]:
A.append(7) # this appends "7" to A
print(A)
# what happens if you evaluate this cell multiple times?

In [ ]:
A = A[:5] # let's set A back to how it was.
print(A)

In [ ]:
A = A + ["cat"]  # Python is totally cool with this
print(A)

In [ ]:
A = [6,4,3,8,5]
for x in A:  # we can iterate over items in a list to get a for loop
    print(2*x)

In [ ]:
# Notice that there's no {} or ; or anything like that.  
#Python uses the whitespace to tell what's in the loop and what's not.

for x in A:
    print(3*x)
print("This is outside the loop")

print("---")

for x in A:
    print(3*x)
    print("This is inside the loop")

In [ ]:
T = range(5)  # the range function gives you a way to iterate over a range of integers
for x in T:
    print(x)

In [ ]:
for i in range(5):  # we can also use the range function to iterate over A
    print(2*A[i])

In [ ]:
for i in range(len(A)):  # and if we don't know how long A is to begin with, we can just use len(A)
    print(2*A[i])

In [ ]:
B = [] # make an empty list
for x in A:
    B.append(2*x)  
print(B)

In [ ]:
C = [ 2*x for x in A ] 
# This makes exactly the same list B that we had before, but in just one line.
print(C)


In [ ]:
def f(x,y):  # this is how we define a function.  Notice that x and y don't have types.
    return x + y

print(f(2,3))  # python has one version of + for integers
print(f([1,2,3],[4,5,6]))  # and another version for lists
print(f("hello ", "world"))  # and another version for strings
# what happens if you do f(2, "cat")?

### As a more serious pass, here is a nice tutorial: https://www.programiz.com/python-programming

### For now you just need to be able to understand (and maybe slightly modify) other people's python code.  

## Let's setup the SQL environment 



In [ ]:
#Install pysqlite3 for python and import pandas to use later
!pip install pysqlite3
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML

: 

Let's define some helper functions for running queries and printing results

In [ ]:
dbname = "music_streaming4.db"

def printSqlResults(cursor, tblName):
  try:
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML("<b><font color=Green> " + tblName + "</font></b>" + df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname) # Connect to the database
  cursor = conn.cursor() # Create a cursor (think: it's like a "pointer")
  cursor.execute(query) # Execute the query
  printSqlResults(cursor, caption) # Print the results
  conn.close()

def runStepByStepSql(query, fromline):
  lines = query.strip().split('\n')
  for lineidx in range(fromline, len(lines)):
    partial_query = '\n'.join(lines[:lineidx])
    caption = 'Query till line:' +  partial_query
    runSql(caption, partial_query + ';')

Let's setup a Schema and insert some data

In [ ]:
# Connect to database (creates the file if it doesn't exist)
"""
1. Connections: A connection represents a connection to a database through
which we can execute SQL queries. The dbname here specifies the database.
In SQLlite, if the DB doesn't exist, it will be created.
2. Cursors: A cursor is an object associated with a database connection.
It allows you to execute SQL queries, fetch query results.
"""
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

# Create the Users table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL UNIQUE
);
""")

# Create the Songs table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
);
""")

# Create the Listens table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Listens (
    listen_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    song_id INTEGER NOT NULL,
    rating FLOAT,
    listen_time TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (song_id) REFERENCES Songs(song_id)
);
""")

# Create the recommendations table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Recommendations (
    user_id INTEGER NOT NULL,
    song_id INTEGER NOT NULL,
    recommendation_id not NULL,
    recommendation_time TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (song_id) REFERENCES Songs(song_id)
);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

In [ ]:
# Connect to database again and insert sample data
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor = conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

# Insert sample users
cursor.execute("""
INSERT INTO Users (user_id, name, email)
VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")

# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating)
VALUES
    (1, 1, 1, 4.5),
    (2, 1, 2, 4.2),
    (3, 1, 6, 3.9),
    (4, 2, 2, 4.7),
    (5, 2, 7, 4.6),
    (6, 2, 8, 3.9),
    (7, 3, 1, 2.9),
    (8, 3, 2, 4.9),
    (9, 3, 6, NULL);
""")
# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")